In [1]:
from src import environments
import nest_asyncio
nest_asyncio.apply()

from gym_derk.envs import DerkEnv
import gym_derk
from tensorflow.keras import layers
from tensorflow import keras
import numpy as np
from src.environments import EnvPerso, EnvPersoInput

2023-06-29 20:08:30.291144: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
%load_ext autoreload
%autoreload 2

In [26]:
reward= {
    "damageEnemyStatue": 4,
    "damageEnemyUnit": 2,
    "killEnemyStatue": 4,
    "killEnemyUnit": 2,
    "healFriendlyStatue": 1,
    "healTeammatel": 2,
    "healTeammate2": 2,
    "timeSpentHomeBase": 0,
    "timeSpentHomeTerritory": 0,
    "timeSpentAwayTerritory": 0,
    "timeSpentAwayBase": 0,
    "damageTaken": -1,
    "friendlyFire": -1,
    "healEnemy": -1,
    "fallDamageTaken": -10,
    "statueDamageTaken": 0,
    "manualBonus": 0,
    "victory": 100,
    "loss": -100,
    "tie": 0,
    "teamSpirit": 0.5,
    "timeScaling": 1,
}

env = EnvPersoInput(reward_function = reward)

In [22]:
env.reset()
action = np.random.uniform(-1, 1, size=(15,))
a = env.step(action)

In [38]:
type(a[0])

numpy.ndarray

In [29]:
env.close()

In [6]:

import tensorflow as tf
from tf_agents.agents.ppo import ppo_agent
from tf_agents.environments import suite_gym
from tf_agents.networks import actor_distribution_network, value_network
from tf_agents.utils import common

# Define the network for the agent.
actor_net = actor_distribution_network.ActorDistributionNetwork(
    env.observation_spec(),
    env.action_spec())

value_net = value_network.ValueNetwork(
    env.observation_spec())

# Define the keras optimizer.
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

# Define the train step counter.
train_step_counter = tf.Variable(0)

# Create the PPO agent.
agent = ppo_agent.PPOAgent(
    env.time_step_spec(),
    env.action_spec(),
    optimizer,
    actor_net=actor_net,
    value_net=value_net,
    train_step_counter=train_step_counter)

# Initialize the agent.
agent.initialize()

# Now you can use this agent to interact with the environment and train it.

In [13]:
iterations = 1000
env.batch_size = 32
learning_rate = 0.001
nb_episodes = 2

In [14]:
from tf_agents.replay_buffers import tf_uniform_replay_buffer

replay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(
                data_spec=agent.collect_data_spec,
                batch_size=env.batch_size,
                max_length=100)

In [15]:
# import episode drive from tf_agents
from tf_agents.drivers import dynamic_episode_driver

collect_driver = dynamic_episode_driver.DynamicEpisodeDriver(
    env,
    agent.collect_policy,
    observers=[replay_buffer.add_batch],
    num_episodes=nb_episodes)

In [24]:
def compute_avg_return(env, policy, num_episodes=10):

  total_return = 0.0
  for _ in range(num_episodes):

    obs = env.reset()
    episode_return = 0.0
    done = False
    
    while not done:
      action_step = policy.action(obs)
      obs, reward, done, info = env.step(action_step.action)
      episode_return += reward
    total_return += episode_return

  avg_return = total_return / num_episodes
  return avg_return.numpy()[0]

In [27]:
from tf_agents.policies import random_tf_policy

random_policy = random_tf_policy.RandomTFPolicy(env.time_step_spec(),
                                                env.action_spec())

In [28]:
compute_avg_return(env, random_policy, num_episodes=2)

/home/trist/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/tf_agents/policies/tf_policy.py:253: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if policy_state is ():  # pylint: disable=literal-comparison
/home/trist/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/tf_agents/policies/tf_policy.py:315: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if not (policy_state is None or policy_state is () or policy_state is []):  # pylint: disable=literal-comparison
/home/trist/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/tf_agents/policies/tf_policy.py:253: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if policy_state is ():  # pylint: disable=literal-comparison
/home/trist/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/tf_agents/policies/tf_policy.py:315: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if not (policy_state is None or policy_state is () or policy_state is []):  # pyl

ValueError: time_step and time_step_spec structures do not match:
  .
vs.
  TimeStep(
{'discount': ., 'observation': ., 'reward': ., 'step_type': .})
Values:
  [ 1.          1.          0.2657708  -0.80199313  0.19179827 -0.321679
  0.33222237 -0.40991098  1.         -0.047903    0.8384946  -0.09466945
  0.68069977 -0.06700578  0.6708463  -0.00992829  0.          0.
  0.          0.          0.          1.          1.          0.
  0.          0.11146335  0.11146335  0.11146335 -0.05085628 -0.13218826
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          1.          0.
  0.          0.          1.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.        ]
vs.
  TimeStep(
{'discount': BoundedTensorSpec(shape=(), dtype=tf.float32, name='discount', minimum=array(0., dtype=fl

In [ ]:
# class Agent:
#     def __init__(self, shape_of_model: tuple):
#         nb_outputs = 15
#         self.model = keras.models.Sequential()
#         self.model.add(layers.Input(shape=(64,)))
#         for n in shape_of_model:
#             self.model.add(layers.Dense(units=n, activation='relu'))
#         self.model.add(layers.Dense(units=nb_outputs, activation='linear'))
    
#     def action(self, state: np.ndarray):
#         prediction = self.model.predict(state)
#         # min max scaling
        
#         return self.model.predict(state)
    


In [ ]:
# class AgentManager:
#     def __init__(self, env, number_of_agents):
#         self.env = env
#         self.agents = [Agent(shape_of_model=(15,15)) for i in range(number_of_agents)]
            
#     def take_action(self, observation):
#         return [agent.action(observation) for agent in self.agents]   
        
        